In [ ]:
import numpy as np
from matplotlib import pyplot as plt
import numpyro
from numpyro.infer import MCMC, NUTS, Predictive
import numpyro.distributions as dist
from numpyro.diagnostics import hpdi
from jax import random
import jax

from jax.scipy.stats import norm
from jax.scipy.special import ndtri
#assert numpyro.__version__.startswith("0.11.0")

numpyro.set_host_device_count(4)
jax.config.update('jax_enable_x64',True)


import jax_cosmo as jc
from jax_cosmo import Cosmology, background

from astropy.cosmology import FlatLambdaCDM,wCDM
from astropy.coordinates import Distance

from numpyro import sample

import numpyro.distributions as dist

import jax.numpy as jnp
from jax import grad, jit, vmap, value_and_grad
from numpyro.distributions.truncated import TruncatedDistribution 
from numpyro.distributions import (
    Distribution,
    constraints)
from numpyro.distributions.util import is_prng_key, promote_shapes, validate_sample
from jax import lax
from SkewNormalPlus import SkewNormalPlus

In [ ]:

import numpyro
from NSF import NeuralSpline1D
from flow import Normal,Flow,transform,Serial
from SkewNormalPlus import SkewNormalPlus as snp

@jit
def minmax_fit_and_scale(X):
  max= jnp.max(X,axis=0)
  min = jnp.min(X,axis=0)
  X_std = (X - min) / (max-min)
  return X_std,min,max

@jit
def minmax_scale(X,min,max):
  return (X - min) / (max - min)

@jit
def minmax_unscale(X,min,max):
  return X * (max - min) + min

from jax.example_libraries import stax, optimizers

from jax.example_libraries.stax import (Dense, Tanh, Flatten, Relu, LogSoftmax, Softmax, Exp,Sigmoid,Softplus,LeakyRelu)

def network(rng,conditional_dim,out_dim, hidden_dim):
    init_fun,apply_fun=stax.serial(stax.Dense(hidden_dim), Relu, stax.Dense(1024),  Relu, stax.Dense(1024),Relu,stax.Dense(hidden_dim), Relu, stax.Dense(out_dim),)
    _, params = init_fun(rng, (conditional_dim,))

    return params,apply_fun

rng, flow_rng = random.split(random.PRNGKey(0))

init_fun = Flow(Serial(*(NeuralSpline1D(network,hidden_dim=256,K=40,B=5),)*5),prior=Normal())

_, log_pdf_flow, sample_flow = init_fun(flow_rng,2 )



In [ ]:

file = np.load('flow2d.npz',allow_pickle=True)
params_file = file['params']
min_file = jnp.asarray(file['min'])
max_file = jnp.asarray(file['max'])

class FlowSNP(Distribution):
    arg_constraints = {"m_int": constraints.real, "sigma_int": constraints.real,"m_cut": constraints.real, "sigma_cut": constraints.real}
    support = constraints.real
    reparametrized_params = ["m_int", "sigma_int","m_cut","sigma_cut"]

    def __init__(self,m_int,sigma_int,m_cut,sigma_cut,*, validate_args=None,res=1000):
        
        
        self.params = params_file
        self.min = min_file
        self.max = max_file

        

        
        self.m_int, self.sigma_int,self.m_cut,self.sigma_cut = promote_shapes(m_int,sigma_int,m_cut,sigma_cut)


        batch_shape = lax.broadcast_shapes(jnp.shape(m_int), jnp.shape(sigma_int),jnp.shape(m_int), jnp.shape(sigma_int))
        
        n_obj = m_int.shape[0]
        
        
        self.n_obj=n_obj
        


        super(FlowSNP, self).__init__(
            batch_shape=batch_shape, validate_args=validate_args
        )
        


    def sample(self,key, sample_shape=()):
        assert is_prng_key(key)
        
        
        
        no_samps = sample_shape[0]
        
        self.no_samps = no_samps
        
        theta = jnp.column_stack((jnp.repeat(self.m_int.reshape(self.n_obj),no_samps).reshape(no_samps*self.n_obj,1),jnp.repeat(self.sigma_int,no_samps*self.n_obj).reshape(no_samps*self.n_obj,1)))
        
        samp=sample_flow(key, self.params,minmax_scale(theta,self.min[1:],self.max[1:]),num_samples=self.no_samps*self.n_obj)
        
        samp = minmax_unscale(samp[:,0],self.min[0],self.max[0])
        
        
        
        return samp.reshape(self.n_obj,self.no_samps).T

    
    
    def log_prob(self, value):
        
        no_samps = value.shape[0]
        no_obj = value.shape[1]
        
        
        value=value.T.reshape(no_samps*no_obj,)
        value = value.reshape(no_samps*no_obj,1)
        
        
        
        theta=jnp.column_stack((self.m_int.reshape(no_obj,1),jnp.repeat(self.sigma_int,no_obj).reshape(no_obj,1)))       
        
        theta = np.repeat(theta,no_samps,axis=0)

        
        X=minmax_scale(jnp.column_stack((value,theta)),self.min,self.max)
            
        
        prob = log_pdf_flow(self.params,X)
        prob = prob.reshape(no_obj*no_samps,)
        prob=prob.reshape(no_obj,no_samps).T
        

        return prob



In [ ]:


def sample_model(z_s,mag_s=None,sel_s=None,M0=-19.5,sigma_int=0.1,h=0.7324,Om0=0.28,w=-1,mu_cut=21,sigma_cut=0.01):

    cosmo_jax = Cosmology(Omega_c=Om0, h=h, w0=w, Omega_b=0, n_s= 0.96, sigma8=200000, Omega_k=0, wa=0)

    
    n_sne=len(z_s)
    
    with numpyro.plate("plate_i",n_sne):
        
        
        d_s=background.transverse_comoving_distance(cosmo_jax, 1/(1+z_s))
    
        mu_s=5*jnp.log10((1+z_s)/h*d_s*1e6/10)
        

        
        m_s=numpyro.sample("m_s", dist.Normal(M0+mu_s, sigma_int), obs=mag_s)
        
        p_s=norm.cdf(-m_s,loc=-mu_cut,scale=sigma_cut)
        
        sel_s=numpyro.sample("sel_s",dist.Bernoulli(p_s),obs=sel_s)
        

def model_flat(z_s,m_s=None,sel_s=None,h=0.7324,w=-1,mu_cut=21,sigma_cut=0.01):

    
    M0=numpyro.sample('M0', dist.ImproperUniform(dist.constraints.real,(),event_shape=()))
    sigma_int = numpyro.sample("sigma_int", dist.HalfNormal(0.2))
    Om0=numpyro.sample('Om0',dist.Uniform(0,1))
    
    cosmo_jax = Cosmology(Omega_c=Om0, h=h, w0=w, Omega_b=0, n_s= 0.96, sigma8=200000, Omega_k=0, wa=0)

    
    n_sne=len(z_s)
    
    with numpyro.plate("plate_i",n_sne):
        
        
        d_s=background.transverse_comoving_distance(cosmo_jax, 1/(1+z_s))
    
        mu_s=5*jnp.log10((1+z_s)/h*d_s*1e6/10)
        
        m_0 = M0 + mu_s
        
        m_s=numpyro.sample("m_s", FlowSNP(m_int=m_0, sigma_int=sigma_int,
                                                m_cut=mu_cut,sigma_cut=sigma_cut), obs=m_s)
        
        #m_s=numpyro.sample("m_s", SkewNormalPlus(m_int=m_0, sigma_int=sigma_int,
         #                                       m_cut=mu_cut,sigma_cut=sigma_cut), obs=m_s)
        
        
  

        
def naive_sample_model(z_s,mag_s=None,M0=-19.5,sigma_int=0.1,h=0.7324,Om0=0.28,w=-1):

    cosmo_jax = Cosmology(Omega_c=Om0, h=h, w0=w, Omega_b=0, n_s= 0.96, sigma8=200000, Omega_k=0, wa=0)

    n_sne=len(z_s)
    
    with numpyro.plate("plate_i",n_sne):
        
        
        d_s=background.transverse_comoving_distance(cosmo_jax, 1/(1+z_s))
    
        mu_s=5*jnp.log10((1+z_s)/h*d_s*1e6/10)
        
        m_s=numpyro.sample("m_s", dist.Normal(M0+mu_s, sigma_int), obs=mag_s)
        
        
        
def naive_model_flat(z_s,m_s=None,h=0.7324,w=-1):

    
    M0=numpyro.sample('M0', dist.ImproperUniform(dist.constraints.real,(),event_shape=()))
    sigma_int = numpyro.sample("sigma_int", dist.HalfCauchy(1))
    Om0=numpyro.sample('Om0',dist.Uniform(0,1))
    
    cosmo_jax = Cosmology(Omega_c=Om0, h=h, w0=w, Omega_b=0, n_s= 0.96, sigma8=200000, Omega_k=0, wa=0)

    
    n_sne=len(z_s)
    
    with numpyro.plate("plate_i",n_sne):
        
        
        d_s=background.transverse_comoving_distance(cosmo_jax, 1/(1+z_s))
    
        mu_s=5*jnp.log10((1+z_s)/h*d_s*1e6/10)
        
        m_s=numpyro.sample("m_s", dist.Normal(M0+mu_s, sigma_int), obs=m_s)
        


        
u=np.random.rand(100000)

#z_s=jnp.asarray((3*u)**(1/3)/(3)**(1/3)*1.2)

z_s = jnp.array(u*(0.3-0.215) + 0.215)

seed=1701

rng_key = jax.random.PRNGKey(seed)
rng_key, rng_key_ = random.split(rng_key)
prior_predictive = Predictive(sample_model, num_samples=1)
prior_predictions = prior_predictive(rng_key_,z_s)

m_sim = prior_predictions["m_s"][0,:]
sel_sim = prior_predictions["sel_s"][0,:]

plt.figure(figsize=(8,6))
plt.scatter(z_s,m_sim,color='blue',label='All')
plt.scatter(z_s[sel_sim==1],m_sim[sel_sim==1],color='red',label='Observed')
plt.ylabel('$m$')
plt.xlabel('$z$')
plt.legend()
plt.show()
print(len(m_sim[sel_sim==1]))

In [ ]:
plt.hist(z_s,bins=100,density=True)
plt.hist(z_s[sel_sim==1],bins=100,density=True)
plt.show()

In [ ]:
print(z_s.shape)
print(sel_sim[sel_sim==1].shape)

In [ ]:
nuts_kernel = NUTS(model_flat,adapt_step_size=True)
#nuts_kernel = NUTS(model_flat,adapt_step_size=True)
mcmc = MCMC(nuts_kernel, num_samples=1000, num_warmup=1000,num_chains=4)
rng_key = random.PRNGKey(0)
#mcmc.run(rng_key, z_s,m_s= m_sim)




#mcmc.run(rng_key, z_s[sel_sim==1.],m_s= m_sim[sel_sim==1.],sel_s=sel_sim[sel_sim==1.])


mcmc.run(rng_key, z_s[sel_sim==1.],m_s= m_sim[sel_sim==1.].reshape(1,len( m_sim[sel_sim==1.])))




In [ ]:
import corner
mcmc.print_summary()
posterior_samples = mcmc.get_samples()
a=['M0', 'Om0', 'sigma_int']


samps=np.zeros((len(a),4000))

for i in range(len(a)):
    samps[i,:]=posterior_samples[a[i]]


corner.corner(samps.T,labels=a,truths=[-19.5,0.28,0.1])
plt.show()

In [ ]:

nuts_kernel = NUTS(naive_model_flat,adapt_step_size=True)
mcmc2 = MCMC(nuts_kerne
             l, num_samples=1000, num_warmup=1000,num_chains=4)
rng_key = random.PRNGKey(0)
#mcmc.run(rng_key, z_s,m_s= m_sim)
mcmc2.run(rng_key, z_s[sel_sim==1.],m_s= m_sim[sel_sim==1.])


In [ ]:
import corner
mcmc2.print_summary()
posterior_samples2 = mcmc2.get_samples()
a=['M0', 'Om0', 'sigma_int']


samps2=np.zeros((len(a),4000))

for i in range(len(a)):
    samps2[i,:]=posterior_samples[a[i]]


corner.corner(samps2.T,labels=a,truths=[-19.5,0.28,0.1])
plt.show()

In [ ]:
import corner
import matplotlib.lines as mlines
import matplotlib 

matplotlib.rcParams.update({'font.size': 14})




a=['M0','Om0','sigma_int']

posterior_samples = mcmc.get_samples()


samps=np.zeros((len(a),4000))

for i in range(len(a)):
    samps[i,:]=posterior_samples[a[i]]


truths_arr=[-19.5,0.28,0.1]



fig=corner.corner(samps.T,truths=truths_arr,smooth=0.9,
    label_kwargs=dict(fontsize=35),
    title_kwargs=dict(fontsize=16),
    levels=(1 - np.exp(-0.5), 1 - np.exp(-2), 1 - np.exp(-9 / 2.)),
    plot_density=False,
    plot_datapoints=False,
    fill_contours=False,
    show_titles=False,
    max_n_ticks=3,truth_color='black',color='green',labelpad=0.15,title_fmt='.3f')


posterior_samples2 = mcmc2.get_samples()


samps2=np.zeros((len(a),4000))

for i in range(len(a)):
    samps2[i,:]=posterior_samples2[a[i]]



corner.corner(samps2.T,labels=[r'$M_0$',r'$\Omega_{m0}$',r'$\sigma_{int}$'],truths=truths_arr,smooth=0.9,
    label_kwargs=dict(fontsize=15),
    title_kwargs=dict(fontsize=15),
  
    levels=(1 - np.exp(-0.5), 1 - np.exp(-2), 1 - np.exp(-9 / 2.)),
    plot_density=False,
    plot_datapoints=False,
    fill_contours=False,
    show_titles=False,
    max_n_ticks=3,color='red',truth_color='black',labelpad=0.15,title_fmt='.3f',fig=fig)


red_line = mlines.Line2D([], [], color='red', label=r'No Selection Model',linewidth=10)
green_line = mlines.Line2D([], [], color='green', label=r'CDF Selection Model',linewidth=10)
plt.legend(handles=[red_line,green_line],fontsize=15, frameon=False,bbox_to_anchor=(1, 2.5, 0.1, 0.1),loc='upper right')
plt.savefig('panth_corner.pdf',bbox_inches='tight',pad_inches=0.1)
plt.show()

In [ ]:
mcmc.print_summary()
mcmc2.print_summary()

In [ ]:
from astropy.cosmology import FlatLambdaCDM
from astropy.coordinates import Distance

sne=1000

u=np.random.rand(sne)

red=(3*u)**(1/3)/(3)**(1/3)*1.2

noise=np.random.normal(0,0.1,sne)


cosmo = FlatLambdaCDM(H0=73.24, Om0=0.28)




d=np.array(Distance(z=red, cosmology=cosmo))     

mu=5*np.log10(d*1e6/10)+noise



plt.scatter(red,mu,label='$\Omega_{m0}=0.28,\Omega_{\Lambda 0}=0.72$')



cosmo = FlatLambdaCDM(H0=73.24, Om0=1)
d=np.array(Distance(z=red, cosmology=cosmo)) 
mu=5*np.log10(d*1e6/10)+noise     
plt.scatter(red,mu,label='$\Omega_{m0}=1,\Omega_{\Lambda 0}=0$')


cosmo = FlatLambdaCDM(H0=73.24, Om0=0)

d=np.array(Distance(z=red, cosmology=cosmo)) 
mu=5*np.log10(d*1e6/10)+noise     
       
plt.scatter(red,mu,label='$\Omega_{m0}=0,\Omega_{\Lambda 0}=1$')
plt.xlabel('Redshift')
plt.ylabel('$\mu$')
plt.legend()
plt.show()

In [ ]:
plt.hist(red)

In [ ]:
from scipy.stats import norm

x=np.linspace(-0,40,100)
norm.ppf(0.025)





plt.plot(x,norm.cdf(-x,loc=-23,scale=1))



plt.show()

In [ ]:
from astropy.cosmology import FlatLambdaCDM
from astropy.coordinates import Distance

sne=10000

u=np.random.rand(sne)

red=(3*u)**(1/3)/(3)**(1/3)*1.2

noise=np.random.normal(0,0.1,sne)


cosmo = FlatLambdaCDM(H0=73.24, Om0=0.28)

plt.title(r'$\Omega_{m0}=0.28,\Omega_{\Lambda 0}=0.72$')


d=np.array(Distance(z=red, cosmology=cosmo))     

mu=5*np.log10(d*1e6/10)+noise


m=-19.5+mu


prob_sel=norm.cdf(-m,loc=-22,scale=1)
print(m)

draw=np.random.rand(sne)

sel=np.zeros(sne)

sel[draw<prob_sel]=1

sel2=m<22


plt.scatter(red,mu,label='All',color='black',s=2)

plt.scatter(red[sel==1],mu[sel==1],label=r'Selected $\Phi(22,1)$',color='blue',s=2)
plt.scatter(red[sel2==1],mu[sel2==1],label=r'Selected $\Theta(22)$',color='red',s=2)


plt.xlabel(r'Redshift  $z_s$')
plt.ylabel('Distance Modulus $\mu_s$')
plt.legend(frameon=False)
plt.savefig('toyselec.pdf')
plt.show()

In [ ]:
bins=np.linspace(0,1.2,50)
#plt.hist(red,bins=bins)
plt.hist(red[sel==1],bins=bins)
plt.show()